In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
country_data = pd.read_csv('/kaggle/input/unsupervised-learning-on-country-data/Country-data.csv')
data_dictionary = pd.read_csv('/kaggle/input/unsupervised-learning-on-country-data/data-dictionary.csv')
country_data.head()

In [ ]:
country_data.shape

In [ ]:
country_data.isnull().sum()

In [ ]:
country_data.info()

# **Data Visualization**

In [ ]:
dataset = country_data.iloc[:,1:]
dataset.head()

In [ ]:
# Importing necessary libraries for Visualization
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(8,8))
corr = dataset.corr()
sns.heatmap(dataset.corr(),annot=True)
plt.show()

In [ ]:
pos_ =  corr[corr >= 0.80]
neg_ = corr[corr <= -0.80]
print(pos_)
print(neg_)

**As observed, more that 80 % data are correlated for below features:**
1. Negatively Correlated : "child_mort" <=> "life_expec"
2. Positively Correlated : "income" <=> "gdpp" and "child_mort" <=> "total_fer"

In [ ]:
sns.color_palette()


In [ ]:
sns.set_theme(style="ticks")
sns.pairplot(dataset,palette='red')

KMeans, HSCAN, DBSCAN

1. Choose Number of Clusters i.e. k.
2. Valtidate the chosen k using Clustering Validation : Sillhoute

In [ ]:
X = dataset

In [ ]:
#elbow method
from sklearn.cluster import KMeans
wcss=[]
K = range(1,11)
for i in K:
    kmeans=KMeans(n_clusters=i,init='k-means++',random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(K,wcss,marker='o')
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

As per Elbow graph, "3" should be the Number of Cluster.
Lets validate using Silhoute Score

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm


range_n_clusters = [2,3,4,5,6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 1 column
    fig, (ax1) = plt.subplots(1)
    fig.set_size_inches(10, 5)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

**As observed For cluster 2, we've highest Silhouette Score but it do have negative values so, it can't be consider.
Hence, 3 is the optimal value of K.**

Now Apply K-Means clustering with number of cluster = 3

In [ ]:
# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(X)
print(y_kmeans)

In [ ]:
# Count of records in each cluster
pd.Series(kmeans.labels_).value_counts()

In [ ]:
pd.options.display.max_rows = None
ClusterNumber = pd.DataFrame(kmeans.labels_)
Country = country_data['country']
df1 = pd.concat([ClusterNumber,Country],axis=1)
df1
